In [4]:
import os
import sys
import cv2
import random
import warnings

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow import keras

from tqdm import tqdm
from skimage.io import imread, imshow
from skimage.transform import resize

In [3]:
# Set some parameters
IMG_WIDTH = 128
IMG_HEIGHT = 128
IMG_CHANNELS = 3

In [5]:
PATH_IMAGES_TRAIN = 'data/training_segmentation/train/images'
PATH_LABELS_TRAIN = 'data/training_segmentation/train/labels'
PATH_IMAGES_TEST = 'data/training_segmentation/test/images'
PATH_LABELS_TEST = 'data/training_segmentation/test/labels'

In [6]:
warnings.filterwarnings('ignore', category=UserWarning, module='skimage')
seed = 42
random.seed(seed)
np.random.seed(seed)
tf.random.set_seed(seed)

In [7]:
# Helper functions
def load_image(image_path, mask=False):
    img = tf.io.read_file(image_path)
    if mask:
        img = tf.image.decode_jpeg(img, channels=1)
        img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
        img = tf.cast(img, tf.float32) / 255.0
    else:
        img = tf.image.decode_jpeg(img, channels=3)
        img = tf.image.resize(img, [IMG_HEIGHT, IMG_WIDTH])
        img = tf.cast(img, tf.float32) / 255.0
    return img

def load_data(image_path, mask_path):
    image = load_image(image_path)
    mask = load_image(mask_path, mask=True)
    return image, mask

In [8]:
# Prepare datasets
train_images = sorted([os.path.join(PATH_IMAGES_TRAIN, fname) for fname in os.listdir(PATH_IMAGES_TRAIN)])
train_masks = sorted([os.path.join(PATH_LABELS_TRAIN, fname) for fname in os.listdir(PATH_LABELS_TRAIN)])

test_images = sorted([os.path.join(PATH_IMAGES_TEST, fname) for fname in os.listdir(PATH_IMAGES_TEST)])
test_masks = sorted([os.path.join(PATH_LABELS_TEST, fname) for fname in os.listdir(PATH_LABELS_TEST)])


In [10]:
# Create tf.data.Dataset
AUTOTUNE = tf.data.experimental.AUTOTUNE
BATCH_SIZE = 32

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_masks))
train_dataset = train_dataset.map(load_data, num_parallel_calls=AUTOTUNE)
train_dataset = train_dataset.shuffle(buffer_size=1000).batch(BATCH_SIZE).prefetch(AUTOTUNE)

test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_masks))
test_dataset = test_dataset.map(load_data, num_parallel_calls=AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(AUTOTUNE)


In [11]:
# Build U-Net model
def unet_model(input_size=(IMG_HEIGHT, IMG_WIDTH, IMG_CHANNELS)):
    inputs = keras.Input(input_size)
    
    # Encoder (Downsampling)
    conv1 = keras.layers.Conv2D(16, 3, activation='elu', padding='same')(inputs)
    conv1 = keras.layers.Dropout(0.1)(conv1)
    conv1 = keras.layers.Conv2D(16, 3, activation='elu', padding='same')(conv1)
    pool1 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv1)
    
    conv2 = keras.layers.Conv2D(32, 3, activation='elu', padding='same')(pool1)
    conv2 = keras.layers.Dropout(0.1)(conv2)
    conv2 = keras.layers.Conv2D(32, 3, activation='elu', padding='same')(conv2)
    pool2 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv2)
    
    conv3 = keras.layers.Conv2D(64, 3, activation='elu', padding='same')(pool2)
    conv3 = keras.layers.Dropout(0.2)(conv3)
    conv3 = keras.layers.Conv2D(64, 3, activation='elu', padding='same')(conv3)
    pool3 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv3)
    
    conv4 = keras.layers.Conv2D(128, 3, activation='elu', padding='same')(pool3)
    conv4 = keras.layers.Dropout(0.2)(conv4)
    conv4 = keras.layers.Conv2D(128, 3, activation='elu', padding='same')(conv4)
    pool4 = keras.layers.MaxPooling2D(pool_size=(2, 2))(conv4)
    
    # Bridge
    conv5 = keras.layers.Conv2D(256, 3, activation='elu', padding='same')(pool4)
    conv5 = keras.layers.Dropout(0.3)(conv5)
    conv5 = keras.layers.Conv2D(256, 3, activation='elu', padding='same')(conv5)
    
    # Decoder (Upsampling)
    up6 = keras.layers.Conv2DTranspose(128, 2, strides=(2, 2), padding='same')(conv5)
    up6 = keras.layers.concatenate([up6, conv4])
    conv6 = keras.layers.Conv2D(128, 3, activation='elu', padding='same')(up6)
    conv6 = keras.layers.Dropout(0.2)(conv6)
    conv6 = keras.layers.Conv2D(128, 3, activation='elu', padding='same')(conv6)
    
    up7 = keras.layers.Conv2DTranspose(64, 2, strides=(2, 2), padding='same')(conv6)
    up7 = keras.layers.concatenate([up7, conv3])
    conv7 = keras.layers.Conv2D(64, 3, activation='elu', padding='same')(up7)
    conv7 = keras.layers.Dropout(0.2)(conv7)
    conv7 = keras.layers.Conv2D(64, 3, activation='elu', padding='same')(conv7)
    
    up8 = keras.layers.Conv2DTranspose(32, 2, strides=(2, 2), padding='same')(conv7)
    up8 = keras.layers.concatenate([up8, conv2])
    conv8 = keras.layers.Conv2D(32, 3, activation='elu', padding='same')(up8)
    conv8 = keras.layers.Dropout(0.1)(conv8)
    conv8 = keras.layers.Conv2D(32, 3, activation='elu', padding='same')(conv8)
    
    up9 = keras.layers.Conv2DTranspose(16, 2, strides=(2, 2), padding='same')(conv8)
    up9 = keras.layers.concatenate([up9, conv1])
    conv9 = keras.layers.Conv2D(16, 3, activation='elu', padding='same')(up9)
    conv9 = keras.layers.Dropout(0.1)(conv9)
    conv9 = keras.layers.Conv2D(16, 3, activation='elu', padding='same')(conv9)
    
    outputs = keras.layers.Conv2D(1, 1, activation='sigmoid')(conv9)
    
    model = keras.Model(inputs=inputs, outputs=outputs)
    return model

In [12]:
# Create and compile the model
model = unet_model()
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 128, 128,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d (Conv2D)     │ (None, 128, 128,  │        448 │ input_layer[0][0] │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout (Dropout)   │ (None, 128, 128,  │          0 │ conv2d[0][0]      │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_1 (Conv2D)   │ (None, 128, 128,  │      2,320 │ dropout[0][0]     │
│                     │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d       │ (None, 64, 64,    │          0 │ conv2d_1[0][0]    │
│ (MaxPooling2D)      │ 16)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_2 (Conv2D)   │ (None, 64, 64,    │      4,640 │ max_pooling2d[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_1 (Dropout) │ (None, 64, 64,    │          0 │ conv2d_2[0][0]    │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_3 (Conv2D)   │ (None, 64, 64,    │      9,248 │ dropout_1[0][0]   │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_1     │ (None, 32, 32,    │          0 │ conv2d_3[0][0]    │
│ (MaxPooling2D)      │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_4 (Conv2D)   │ (None, 32, 32,    │     18,496 │ max_pooling2d_1[… │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_2 (Dropout) │ (None, 32, 32,    │          0 │ conv2d_4[0][0]    │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_5 (Conv2D)   │ (None, 32, 32,    │     36,928 │ dropout_2[0][0]   │
│                     │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_2     │ (None, 16, 16,    │          0 │ conv2d_5[0][0]    │
│ (MaxPooling2D)      │ 64)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_6 (Conv2D)   │ (None, 16, 16,    │     73,856 │ max_pooling2d_2[… │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dropout_3 (Dropout) │ (None, 16, 16,    │          0 │ conv2d_6[0][0]    │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ conv2d_7 (Conv2D)   │ (None, 16, 16,    │    147,584 │ dropout_3[0][0]   │
│                     │ 128)              │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ max_pooling2d_3     │ (None, 8, 8, 128) │          0 │ conv2d_7[0][0]  

 Total params: 1,941,105 (7.40 MB)

 Trainable params: 1,941,105 (7.40 MB)

 Non-trainable params: 0 (0.00 B)

In [14]:
# Callbacks
callbacks = [
    keras.callbacks.ModelCheckpoint('best_model.keras', save_best_only=True),
    keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-6),
    keras.callbacks.EarlyStopping(patience=10, restore_best_weights=True),
    keras.callbacks.TensorBoard(log_dir='./logs')
]

In [15]:
# Train the model
history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=test_dataset,
    callbacks=callbacks
)

Epoch 1/50


2024-08-28 20:04:53.215050: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:450] ShuffleDatasetV3:2: Filling up shuffle buffer (this may take a while): 499 of 1000
2024-08-28 20:04:59.073910: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:480] Shuffle buffer filled.
2024-08-28 20:05:01.226253: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2024-08-28 20:05:01.906416: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.


  1/224 ━━━━━━━━━━━━━━━━━━━━ 1:40:38 27s/step - accuracy: 0.4628 - loss: 0.7032

2024-08-28 20:05:06.551224: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.
2024-08-28 20:05:07.013300: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.


  2/224 ━━━━━━━━━━━━━━━━━━━━ 15:30 4s/step - accuracy: 0.5128 - loss: 0.6859   

2024-08-28 20:05:10.144265: W external/local_tsl/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 67108864 exceeds 10% of free system memory.


 33/224 ━━━━━━━━━━━━━━━━━━━━ 10:45 3s/step - accuracy: 0.7306 - loss: 0.5302

KeyboardInterrupt: 

In [ ]:
# Plot training history
plt.figure(figsize=(12, 4))
plt.subplot(121)
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()

In [ ]:
plt.subplot(122)
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()


In [ ]:
# Evaluate on test set
test_loss, test_accuracy = model.evaluate(test_dataset)
print(f"Test Loss: {test_loss:.4f}")
print(f"Test Accuracy: {test_accuracy:.4f}")

In [ ]:
# Predictions
for images, masks in test_dataset.take(1):
    predictions = model.predict(images)
    for i in range(5):  # Show 5 examples
        plt.figure(figsize=(12, 4))
        plt.subplot(131)
        plt.imshow(images[i])
        plt.title('Input Image')
        plt.subplot(132)
        plt.imshow(np.squeeze(masks[i]), cmap='gray')
        plt.title('True Mask')
        plt.subplot(133)
        plt.imshow(np.squeeze(predictions[i]), cmap='gray')
        plt.title('Predicted Mask')
        plt.show()